In [5]:
from astropy.cosmology import WMAP9, z_at_value
import astropy.units as u

import matplotlib.pyplot as plt
import numpy as np
from   scipy.interpolate import interp1d
import glob
from   astropy.io import ascii

from scipy.interpolate import Rbf
from scipy.interpolate import RegularGridInterpolator
import h5py

import matplotlib as matplotlib
import plotsettings as plotsettings

publishable = plotsettings.Set('Cell')

publishable.set_figsize(0.9, 0.9, aspect_ratio=0.95)
matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
matplotlib.rc('text', usetex=False)

mass_arr  = np.arange(0.0, 10.1, 0.2)
X, Y      = np.meshgrid(mass_arr, mass_arr)

def load_ph_D_data(base_dir):
    files = glob.glob("/Users/Sophie/Desktop/GW_final_calculation/newfolder/aligned_spin_new/"+"/LISA_z*")
    redshift_arr = np.array([])
    for file in files:
        name_split      = file.split("_")
        string_redshift = name_split[-1]
        redshift        = float(string_redshift[0:-4])
        redshift_arr    = np.append(redshift_arr, redshift)
 
    redshift_arr = np.sort(redshift_arr)
    
    LISA_Inspiral_SNR_data       = np.zeros([len(mass_arr),len(mass_arr), len(redshift_arr)])
    LISA_MergerRingdown_SNR_data = np.zeros([len(mass_arr),len(mass_arr), len(redshift_arr)])
    fobs_at_merger_data          = np.zeros([len(mass_arr),len(mass_arr), len(redshift_arr)])
    hc_data                      = np.zeros([len(mass_arr),len(mass_arr), len(redshift_arr)])

    for file in files:
        name_split          = file.split("_")
        string_redshift     = name_split[-1]
        redshift            = float(string_redshift[0:-4])
        data                = ascii.read(file)
        print data
        M1                  = np.log10(np.array(data['col4']))
        M2                  = np.log10(np.array(data['col5']))

        hc                  = np.log10(np.array(data['col7']))
        LISA_Inspiral_SNR   = np.log10(np.array(data['col8']))
        LISA_MergerRingdown = np.log10(np.array(data['col9']))
        fobs_at_merger      = np.log10(np.array(data['col3']))
        
        index_k = (np.abs(redshift_arr-redshift)).argmin()

        for i in np.arange(0, len(M1), 1):
            M1_sel  = M1[i]
            M2_sel  = M2[i]
            index_i = (np.abs(M1_sel-mass_arr)).argmin()
            index_j = (np.abs(M2_sel-mass_arr)).argmin()
            LISA_Inspiral_SNR_data[index_i, index_j, index_k]       = LISA_Inspiral_SNR[i]
            LISA_MergerRingdown_SNR_data[index_i, index_j, index_k] = LISA_MergerRingdown[i]
            hc_data[index_i, index_j, index_k]                      = hc[i]

    return {"LISA_Inspiral_SNR":LISA_Inspiral_SNR_data, \
                "LISA_MergerRingdown_SNR":LISA_MergerRingdown_SNR_data,\
                "fobs_at_merger":fobs_at_merger_data, \
                "hc":hc_data, "redshift_arr":redshift_arr}


#################################################
#################################################
#################################################

base_dir = "/Users/qirong_work/Downloads/aligned_spin_new/"
data = load_ph_D_data(base_dir)

redshift_arr       = data["redshift_arr"]
Inspiral_SNR       = data["LISA_Inspiral_SNR"]
hc                 = data["hc"]
MergerRingdown_SNR = data["LISA_MergerRingdown_SNR"]
fobs_at_merger     = data["fobs_at_merger"]


fig = plt.figure()                                                                                                            
ax  = fig.add_subplot(111)

im  = plt.imshow(hc[:, :, 0], \
                    interpolation='bilinear',\
                    origin='lower', \
                    extent=(0, 10.0, 0, 10.0), \
                    alpha=0.5)

levels = np.arange(-20.0, 4.1, 1.0)

CS = plt.contour(X, Y, hc[:, :, 0], \
                     levels, inline=1, fontsize=10, lw=2)

plt.clabel(CS, levels, inline=1, fmt='%1.1f', fontsize=8)

ax.set_ylim([0.0,   10.0])
ax.set_xlim([0.0,   10.0])
ax.set_ylabel(r'$\rm{M_{2}}$', fontsize='large')
ax.set_xlabel(r'$\rm{M_{1}}$', fontsize='large')

plt.savefig("hc_M1_M2.png", bbox_inches='tight', dpi=300)
plt.close()

#################################################
####### how to use the data matrix to interpolate
####### a merger event given M1, M2 and redhisft.
#######
#################################################

#change the auguments to interpolate the quantity you want
inspiral_snr_interpol = \
    RegularGridInterpolator((mass_arr, mass_arr, redshift_arr), MergerRingdown_SNR)

input_mass1    = [1e6, 1e4, 1e2, 1e1, 1e1, 1e3, 1e4, 1e5, 1e6, 1e7]
input_mass2    = [1e6, 1e5, 1e3, 1e1, 1e2, 1e2, 1e2, 1e2, 1e2, 1e2]
input_redshift = [7.2, 5.6, 2.5, 0.2, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

for i in np.arange(len(input_redshift)):
    print ringdown_snr_interpol([np.log10(input_mass1[i]), np.log10(input_mass2[i]), input_redshift[i]])


#################################################
##### save all data to a hdf5 file ##############
#################################################
h    = h5py.File('PhenomD_model_data.hdf5', 'w')
dset = h.create_dataset('redshift_array',     data=redshift_arr)
dset = h.create_dataset('mass1_array',        data=mass_arr)
dset = h.create_dataset('mass2_array',        data=mass_arr)
dset = h.create_dataset('MergerRingdown_SNR', data=MergerRingdown_SNR)
dset = h.create_dataset('Inspiral_SNR',       data=Inspiral_SNR)
dset = h.create_dataset('fobs_at_merger',     data=fobs_at_merger)

h.close()

    col1        col2    ...        col8              col9      
----------- ----------- ... ----------------- -----------------
0.001032005 4.466788952 ...     55.4981435372 4.54995822539e-08
0.001032005 4.466788952 ...     66.9432787773 8.25930858062e-08
0.001032005 4.466788952 ...     80.0790480847 1.55261691573e-07
0.001032005 4.466788952 ...     95.1072670241   2.993066981e-07
0.001032005 4.466788952 ...     112.315645285 5.90292832256e-07
0.001032005 4.466788952 ...      132.08095234 1.19691528693e-06
0.001032005 4.466788952 ...     154.865322743 2.51173935866e-06
0.001032005 4.466788952 ...     181.214540103 5.47376620222e-06
0.001032005 4.466788952 ...     211.761981013 1.23685269916e-05
0.001032005 4.466788952 ...     247.238253058 2.88210783914e-05
        ...         ... ...               ...               ...
0.001032005 4.466788952 ... 1.13360465202e-07 1.30738787982e-05
0.001032005 4.466788952 ... 1.40172379839e-07 1.83242096903e-05
0.001032005 4.466788952 ... 1.7122698507

In [8]:
inspiral_snr_interpol([np.log10(30),np.log10(30), 20]) 

array([-0.63782459])